In [1]:
import pandas as pd
import numpy as np
from astropy.utils.console import ProgressBar

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from music.models import MusicGadgetIc, MusicEllipsoidRegion
from rockstar.models import RockstarCatalogue
from core.models import EllipsoidRegion, Category

In [3]:
from core.utils import create_ellipsoid_region

In [4]:
from music.utils import create_ic_from_existing

In [5]:
# df_selected = pd.read_hdf('selected.hdf5', key='2017-07-20 19:24:04.184707')
df_selected = pd.read_hdf('../media/selected.hdf5', key='2017-07-24 20:16:20.495943')

In [6]:
sim = Simulation.objects.get(name='out512')
snap = sim.snapshot_set.get(snap_number=14)
cat = snap.catalogue_set.get()
halos = cat.structure_set
ic = sim.ic

In [7]:
category = Category.objects.get(name='selection 2')

In [9]:
selected_ids = df_selected.index[28:]
selected_ids

Int64Index([21720], dtype='int64', name='hid')

In [10]:
# create regions

for hid in selected_ids:
    structure = halos.get(rockstarhalo__hid=hid)

    # create EllipsoidRegion
    region = create_ellipsoid_region(structure, rtb=10.0)
    region.category.add(category)
    region.save()
    
    # region = regions.get(structure=structure)
        
    # create MusicEllipsoidRegion
    region = MusicEllipsoidRegion(region=region)
    region.save()

In [19]:
halo_ids = df_selected.index
halo_ids = halo_ids.drop([33306, 124429])
halo_ids


Int64Index([ 79956,  33296,  95915, 116774, 241738, 241466, 206641, 229533,
            124429,  32879,  70495,  47855, 147062, 136254, 136001, 206312,
             79392, 193602,   1330, 146616, 123930,  21971, 135915, 181619,
            228167,    641, 146457,  21720],
           dtype='int64', name='hid')

In [21]:
category = Category.objects.get(name='selection 2')

level = 12
if level==12:
    spreadcoarse = False
else:
    spreadcoarse = True

with ProgressBar(len(halo_ids), ipython_widget=True) as pbar:
    for hid in halo_ids:
        
        # category = root_cat.children.get(name=str(hid))
        # category = category.children.get(name='DM')
        
        new_ic = create_ic_from_existing(ic)
        
        structure = halos.get(rockstarhalo__hid=hid)
        region = Region.objects.get(structure=structure)
        region = MusicEllipsoidRegion.objects.get(region=region)
        
        new_ic.region = region
        new_ic.levelmin = 8
        new_ic.levelmax = level
        
        new_ic.padding = 6
        new_ic.overlap = 4
        
        new_ic.gadget_spreadcoarse = spreadcoarse
        new_ic.category.add(category)
        new_ic.name = '{hid:d}_DM_L{level:d}'.format(hid=hid, level=level)
        new_ic.filename = new_ic.name + '.dat'
        
        new_ic.save()
        
        pbar.update()